# **Cluster South Mumbai**

# **Version 10** 

# **Get required Libraries**

In [0]:
import pandas as pd
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import numpy as np

from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns
from urllib.request import urlopen

import requests
# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Library imported.")


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Library imported 10.


In [0]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print("Library imported.")


Library imported 11.


## **South Mumbai Neighborhoods.**

####  **South Mumbai is most happening borough of Mumbai with lots of tourist attractions. 21 Neighborhoods will go through this analysis.**

In [0]:
areas_south = ["Colaba","Apollo Bandar","Fort","Churchgate","Nariman Point" \
        ,"Marine Lines","Walkeshwar","Malabar Hill","Kalbadevi","Bhuleshwar" \
        ,"Masjid Bandar","Darukhana","Pydhoni","Nagpada","Dongri" \
        ,"Byculla","Grand Road","Mazgaon","Mumbai Central","Mahalaxmi" \
        ,"Worli"]


# **West Mumbai Neighborhoods.**

#### **Nine neighborhoods of West Mumbai selected for analysis.**

In [0]:
areas_west = ["Dadar","Bandra West","Juhu","Andheri","Kalina" \
              ,"Santa Cruz","Mahim","Khar"]
        


# **East Mumbai Neighborhoods.**

#### **Five neighborhoods of East Mumbai selected for analysis.**

In [0]:
areas_east = ["Kurla","Ghatkopar","Chembur","Govandi","Mulund West"] 


# **North Mumbai Neighborhoods.**

#### **Five neighborhoods of North Mumbai selected for analysis.**

In [0]:
areas_north = ["Powai","Jogeshwari","Malad","Borivali","Mira Road"] 


In [0]:
boroughs_south_mumbai = ["South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai"]

          
             

In [0]:
boroughs_west_mumbai = ["West Mumbai","West Mumbai","West Mumbai","West Mumbai","West Mumbai" \
             ,"West Mumbai","West Mumbai","West Mumbai"]
             
            


In [0]:
boroughs_east_mumbai = ["East Mumbai","East Mumbai","East Mumbai","East Mumbai","East Mumbai"]


In [0]:
boroughs_north_mumbai = ["North Mumbai","North Mumbai","North Mumbai","North Mumbai","North Mumbai"]


#### **Note : This split was done due to limitations faced during call to API for retrieving the coordinates of all 39 neighborhoods.**

In [0]:
boroughs1 =  boroughs_south_mumbai
boroughs2 =  boroughs_west_mumbai + boroughs_east_mumbai + boroughs_north_mumbai

In [0]:
areas1 = areas_south
areas2 = areas_west + areas_east + areas_north



### **Building full address to pass to API.**

In [0]:
full_address1 = []
 
for area in areas1:
   
  full_address1.append(area + ", Mumbai")

In [0]:
full_address2 = []
 
for area in areas2:
   
  full_address2.append(area + ", Mumbai")

In [0]:
print("Lenght of areas1={} , boroughs1 = {} , full_address1 = {}" \
.format(len(areas1),len(boroughs1),len(full_address1)))

print("Lenght of areas2={} , boroughs2 = {} , full_address2 = {}" \
.format(len(areas2),len(boroughs2),len(full_address2)))

Lenght of areas1=21 , boroughs1 = 21 , full_address1 = 21
Lenght of areas2=18 , boroughs2 = 18 , full_address2 = 18


## **Create Dictionary first**.

In [0]:
dic_area1 = {"Neighborhood": areas1,"Boroughs": boroughs1,"Address":full_address1}
dic_area2 = {"Neighborhood": areas2,"Boroughs": boroughs2,"Address":full_address2}
print("dic_area1:",dic_area1)
print("dic_area2:",dic_area2)

dic_area1: {'Neighborhood': ['Colaba', 'Apollo Bandar', 'Fort', 'Churchgate', 'Nariman Point', 'Marine Lines', 'Walkeshwar', 'Malabar Hill', 'Kalbadevi', 'Bhuleshwar', 'Masjid Bandar', 'Darukhana', 'Pydhoni', 'Nagpada', 'Dongri', 'Byculla', 'Grand Road', 'Mazgaon', 'Mumbai Central', 'Mahalaxmi', 'Worli'], 'Boroughs': ['South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai'], 'Address': ['Colaba, Mumbai', 'Apollo Bandar, Mumbai', 'Fort, Mumbai', 'Churchgate, Mumbai', 'Nariman Point, Mumbai', 'Marine Lines, Mumbai', 'Walkeshwar, Mumbai', 'Malabar Hill, Mumbai', 'Kalbadevi, Mumbai', 'Bhuleshwar, Mumbai', 'Masjid Bandar, Mumbai', 'Darukhana, Mumbai', 'Pydhoni, Mumbai', 'Nagpada, Mumbai', 'Dongri, Mumbai', 'Byculla, 

## **Create dataframes from dictionary.**

In [0]:
df_area1 = pd.DataFrame.from_dict(dic_area1)
df_area1.head()


,Neighborhood,Boroughs,Address
0,Colaba,South Mumbai,"Colaba, Mumbai"
1,Apollo Bandar,South Mumbai,"Apollo Bandar, Mumbai"
2,Fort,South Mumbai,"Fort, Mumbai"
3,Churchgate,South Mumbai,"Churchgate, Mumbai"
4,Nariman Point,South Mumbai,"Nariman Point, Mumbai"


In [0]:
df_area2 = pd.DataFrame.from_dict(dic_area2)
df_area2.tail()


,Neighborhood,Boroughs,Address
13,Powai,North Mumbai,"Powai, Mumbai"
14,Jogeshwari,North Mumbai,"Jogeshwari, Mumbai"
15,Malad,North Mumbai,"Malad, Mumbai"
16,Borivali,North Mumbai,"Borivali, Mumbai"
17,Mira Road,North Mumbai,"Mira Road, Mumbai"


# **Get coordinates of each neighborhood of Mumbai.**

### **Let's pass South Mumbai dataframe :  df_area1**

#### **Use geopy library to get the latitude and longitude values of Mumbai neigborhoods**
####**In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.**

In [0]:
# South Mumbai first

df2 = df_area1
df2.head()

data = []
for  neighborhood,address,borough in zip(df2['Neighborhood'], df2['Address'],df2['Boroughs']):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    data.append([neighborhood,borough,latitude,longitude])

col_names =  ['Neighborhood', 'Borough', 'Latitude','Longitude']
neighborhoods = pd.DataFrame(data,columns=col_names)

# Assign to neighborhoods1 : South Mumbai
neighborhoods1 = neighborhoods
neighborhoods1.tail()


,Neighborhood,Borough,Latitude,Longitude
16,Grand Road,South Mumbai,18.938771,72.835335
17,Mazgaon,South Mumbai,18.968052,72.840012
18,Mumbai Central,South Mumbai,18.969586,72.819315
19,Mahalaxmi,South Mumbai,18.982568,72.824160
20,Worli,South Mumbai,19.011696,72.818070


In [0]:
print("Coordiates for {} South Mumbai neighborhoods obtained.".format(neighborhoods1.shape[0]) )

Coordiates for 21 South Mumbai neighborhoods obtained.


### **Now let's pass east+west+north Mumabi : df_area2**

In [0]:
df2 = df_area2
df2.tail()

,Neighborhood,Boroughs,Address
13,Powai,North Mumbai,"Powai, Mumbai"
14,Jogeshwari,North Mumbai,"Jogeshwari, Mumbai"
15,Malad,North Mumbai,"Malad, Mumbai"
16,Borivali,North Mumbai,"Borivali, Mumbai"
17,Mira Road,North Mumbai,"Mira Road, Mumbai"


In [0]:
# Now second iteration for Rest of Mumbai.

df2 = df_area2
df2.tail()

data = []
for  neighborhood,address,borough in zip(df2['Neighborhood'], df2['Address'],df2['Boroughs']):
  print("neighborhood:", neighborhood)
  print("address:", address)
  print("borough:", borough)
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  print("latitude:", latitude)
  print("longitude:", longitude)
  data.append([neighborhood,borough,latitude,longitude])

col_names =  ['Neighborhood', 'Borough', 'Latitude','Longitude']
neighborhoods = pd.DataFrame(data,columns=col_names)

#Assign to neighborhoods2 : Rest of Mumbai

neighborhoods2 = neighborhoods
neighborhoods2.tail()

neighborhood: Dadar
address: Dadar, Mumbai
borough: West Mumbai
latitude: 19.019282
longitude: 72.8428757
neighborhood: Bandra West
address: Bandra West, Mumbai
borough: West Mumbai
latitude: 19.0583358
longitude: 72.8302669
neighborhood: Juhu
address: Juhu, Mumbai
borough: West Mumbai
latitude: 19.1070215
longitude: 72.8275275
neighborhood: Andheri
address: Andheri, Mumbai
borough: West Mumbai
latitude: 19.1196976
longitude: 72.8464205
neighborhood: Kalina
address: Kalina, Mumbai
borough: West Mumbai
latitude: 19.079273
longitude: 72.8612672
neighborhood: Santa Cruz
address: Santa Cruz, Mumbai
borough: West Mumbai
latitude: 19.0793694
longitude: 72.8470855
neighborhood: Mahim
address: Mahim, Mumbai
borough: West Mumbai
latitude: 19.0423145
longitude: 72.8398344
neighborhood: Khar
address: Khar, Mumbai
borough: West Mumbai
latitude: 19.0696584
longitude: 72.8398944
neighborhood: Kurla
address: Kurla, Mumbai
borough: East Mumbai
latitude: 19.0652797
longitude: 72.8793805
neighborhood: G

,Neighborhood,Borough,Latitude,Longitude
13,Powai,North Mumbai,19.118720,72.907348
14,Jogeshwari,North Mumbai,19.134899,72.848820
15,Malad,North Mumbai,19.186719,72.848588
16,Borivali,North Mumbai,19.229068,72.857363
17,Mira Road,North Mumbai,19.187896,72.836596


In [0]:
print("Coordiates for {} Rest of Mumbai neighborhoods obtained.".format(neighborhoods2.shape[0]) )

Coordiates for 18 Rest of Mumbai neighborhoods obtained.


## **Now Let's concat two dataframes for clubbing entire Mumbai dataframe to one.** 

In [0]:
neighborhoods = neighborhoods1.append(neighborhoods2,ignore_index=True)
neighborhoods = neighborhoods.reset_index()
neighborhoods.shape


(39, 5)

In [0]:
print("Merged data frame have {} rows.".format(neighborhoods.shape[0]) )
neighborhoods.tail()

Merged data frame have 39 rows.


,index,Neighborhood,Borough,Latitude,Longitude
34,34,Powai,North Mumbai,19.118720,72.907348
35,35,Jogeshwari,North Mumbai,19.134899,72.848820
36,36,Malad,North Mumbai,19.186719,72.848588
37,37,Borivali,North Mumbai,19.229068,72.857363
38,38,Mira Road,North Mumbai,19.187896,72.836596


#### **Let's get coordinates of Mumbai.**

In [0]:
address = "Mumbai"
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
mumbai_latitude = latitude
mumbai_longitude = longitude
print ("Latitude={} and Longitude={}".format(mumbai_latitude,mumbai_longitude))

Latitude=18.9387711 and Longitude=72.8353355


## **Create a map of Mumbai with neighborhoods superimposed on top.**


#### **Here folium library is being used to create city map.**

In [0]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=14)
map_mumbai

# **Add markers to map showing neighborhoods of Mumbai.**

In [0]:
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

# **Four Square API to Get Venues** 

##### Four Square API are being used here to fetch the venues around a location. It needs latitude and longitude of location and returns venues along with categories.

##### **Define Foursquare Credentials and Version**

In [0]:
CLIENT_ID = 'AED1YJAEJO1BSJVJJ5KT2Y2BDGDVPCB21WUXERHTA4B4XXKC' # Foursquare ID
CLIENT_SECRET = '5WF0PKUAGWG30ASKZSBV12XHC2IDIQLEHRRPU541XI2JVQCH' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


credentails:
CLIENT_ID: AED1YJAEJO1BSJVJJ5KT2Y2BDGDVPCB21WUXERHTA4B4XXKC
CLIENT_SECRET:5WF0PKUAGWG30ASKZSBV12XHC2IDIQLEHRRPU541XI2JVQCH


#### **Let's create a function to get nearby venues of each neighborhoods in Mumbai.**

##### Tried to keep an optimum value of radius. With more radius it fetches more vanues but chances of overlapping of neighborhood increases as well.

In [0]:
radius=650
LIMIT = 100
print("The radius is = {} and LIMIT is = {}".format(radius,LIMIT))

def getNearbyVenues(names, latitudes, longitudes, radius=650):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


The radius is = 650 and LIMIT is = 100


##### **Running the above function on each neighborhood and create a new dataframe.**

In [0]:
south_mumbai_data = neighborhoods
south_mumbai_venues = getNearbyVenues(names=south_mumbai_data['Neighborhood'],
                                   latitudes=south_mumbai_data['Latitude'],
                                   longitudes=south_mumbai_data['Longitude']
                                   
                                  )
print("dtypes:",south_mumbai_venues.dtypes)
print("Total venues retrieved:",south_mumbai_venues.shape[0])


Colaba
Apollo Bandar
Fort
Churchgate
Nariman Point
Marine Lines
Walkeshwar
Malabar Hill
Kalbadevi
Bhuleshwar
Masjid Bandar
Darukhana
Pydhoni
Nagpada
Dongri
Byculla
Grand Road
Mazgaon
Mumbai Central
Mahalaxmi
Worli
Dadar
Bandra West
Juhu
Andheri
Kalina
Santa Cruz
Mahim
Khar
Kurla
Ghatkopar
Chembur
Govandi
Mulund West
Powai
Jogeshwari
Malad
Borivali
Mira Road
dtypes: Neighborhood               object
Neighborhood Latitude     float64
Neighborhood Longitude    float64
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
Venue Category             object
dtype: object
Total venues retrieved: 1104


#### **Understand the venue categories well.**

In [0]:
south_mumbai_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Colaba,18.915091,72.825969,Charagh Din,18.915254,72.824151,Men's Store
1,Colaba,18.915091,72.825969,IMBISS Meating Joint,18.917157,72.827018,German Restaurant
2,Colaba,18.915091,72.825969,Thai Pavilion,18.914246,72.821080,Thai Restaurant
3,Colaba,18.915091,72.825969,Theobroma,18.919298,72.829185,Dessert Shop
4,Colaba,18.915091,72.825969,Vivanta by Taj - President,18.914413,72.821028,Hotel


# **Data Cleansing** 

##### Identify the issues with data and rectify these errors like duplicate data etc. in order to make data fit for accurate analysis.

#### **Drop Duplicates Venues**

##### South Mumbai is a narrow strip, so chances of overlap are possible. Therefore, duplicates venues needs to be discarded.

In [0]:
print("rows before dropping duplicates={}".format(south_mumbai_venues.shape[0]))
south_mumbai_venues = south_mumbai_venues.drop_duplicates(subset=['Venue','Venue Latitude', 'Venue Longitude'])
print("rows after dropping duplicates={}".format(south_mumbai_venues.shape[0]))


rows before dropping duplicates=987
rows after dropping duplicates=987


### **Ignore rows with Indian Restaurants**.

##### Indian restaurants are very common and in large number in entire Mumbai. Therefore, decided to get rid of Indian Restaurants by using filter.

In [0]:
print("rows bef filter:",south_mumbai_venues.shape[0])
south_mumbai_venues = south_mumbai_venues[(south_mumbai_venues['Venue Category'] != "Indian Restaurant")].reset_index()
print("rows aft filter:",south_mumbai_venues.shape[0])




rows bef filter: 851
rows aft filter: 851


#### **Unique Categories**

In [0]:
print('There are {} uniques categories.'.format(len(south_mumbai_venues['Venue Category'].unique())))


There are 155 uniques categories.


### **Clubbing Similar Categories**

####For example garden and parks can be clubbed together to gardens/parks etc.

In [0]:
south_mumbai_venues.replace(to_replace =['Movie Theater'],value = 'Theater\Multiplex')
south_mumbai_venues.replace(to_replace =['Theater'],value = 'Theater\Multiplex')
south_mumbai_venues.replace(to_replace =['Multiplex'],value = 'Theater\Multiplex')
south_mumbai_venues.replace(to_replace =['Amphitheater'],value = 'Theater\Multiplex')
south_mumbai_venues.replace(to_replace =['Indie Movie Theater'],value = 'Theater\Multiplex')
south_mumbai_venues.replace(to_replace =['Performing Arts Venue'],value = 'Theater\Multiplex')
south_mumbai_venues.shape

(851, 9)

In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Garden','Park' \
                       ])].shape

(8, 9)

In [0]:
south_mumbai_venues.replace(to_replace =['Garden','Park'],value = 'Park/Garden')
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Park/Garden' \
                       ])].shape
#south_mumbai_venues.shape                       

(851, 9)

In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Pub','Gastropub' \
                             ,'Bar','Cocktail Bar'\
                             ,'Wine Bar','Hotel Bar','Sports Bar'\
                       ])].shape

(65, 9)

In [0]:
south_mumbai_venues.replace(to_replace =['Pub'],value = 'Bar/Pub', inplace=True )
south_mumbai_venues.replace(to_replace =['Gastropub'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Cocktail Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Wine Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Hotel Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Sports Bar'],value = 'Bar/Pub', inplace=True)

In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Bar/Pub' \
                       ])].shape
                     

(65, 9)

In [0]:
south_mumbai_venues.replace(to_replace =['Train Station' \
                                         ,'Bus Station'
                                         ,'Metro Station'
                                         ,'Train'
],value = 'Train/Metro/Bus', inplace=True )

In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Train/Metro/Bus' \
                       ])].shape

(12, 9)

In [0]:
south_mumbai_venues.replace(to_replace =['Spa' \
                                         ,'Gym'
                                         ,'Gym / Fitness Center'
                                         ,'Yoga Studio'
                                         ,'Track'
                                         ,'Massage Studio'
],value = 'Fitness/Gym/Spa', inplace=True )

In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Fitness/Gym/Spa' \
                       ])].shape

(32, 9)

In [0]:
south_mumbai_venues.replace(to_replace =['Tennis Court' \
                                         ,'Athletics & Sports'
                                         ,'Cricket Ground'
                                         ,'Hockey Arena'
                                         ,'Field'
                                         ,'Stadium'
                                         ,'Soccer Field'
                                         ,'Playground'
],value = 'Sports', inplace=True )

In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Venue Category'] \
                      .isin(['Sports' \
                       ])].shape

(14, 9)

In [0]:
### Eliminate neighborhoods with very less venues

In [0]:
df = df[['STNAME','CTYNAME']].groupby(['STNAME'])['CTYNAME'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)


In [0]:
south_mumbai_venues.dtypes

level_0                     int64
index                       int64
Neighborhood               object
Neighborhood Latitude     float64
Neighborhood Longitude    float64
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
Venue Category             object
dtype: object

In [0]:
test_df = south_mumbai_venues
test_df = test_df[['Neighborhood']].groupby(['Neighborhood']) \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)
test_df.shape                             

TypeError: ignored

### **Check neighborhoods with very less venues**

In [134]:
south_mumbai_venues_group_nbhood = south_mumbai_venues
south_mumbai_venues_group_nbhood = south_mumbai_venues_group_nbhood.groupby(['Neighborhood']) \
                             .count() 
south_mumbai_venues_group_nbhood.sort_values(['index'], ascending=True)

,level_0,index,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,,
Pydhoni,1,1,1,1,1,1,1,1
Dongri,2,2,2,2,2,2,2,2
Grand Road,4,4,4,4,4,4,4,4
Kurla,5,5,5,5,5,5,5,5
Jogeshwari,5,5,5,5,5,5,5,5
Govandi,5,5,5,5,5,5,5,5
Byculla,6,6,6,6,6,6,6,6
Kalina,7,7,7,7,7,7,7,7
Malad,7,7,7,7,7,7,7,7


### **Drop Neigborhoods**

##### Drop some neiborhoods with very less venues.

In [133]:
print("before filter:",south_mumbai_venues.shape)
south_mumbai_venues = south_mumbai_venues[south_mumbai_venues.Neighborhood != 'Darukhana']
print("after filter:",south_mumbai_venues.shape)

before filter: (851, 9)
after filter: (850, 9)


### **Merge Neighborhoods**

##### Merge Neigborhoods with very less venues to nearby venues. 

In [161]:
# Get coordinates of Nagpada

Nagpada = south_mumbai_venues
Nagpada = Nagpada[Nagpada.Neighborhood == 'Nagpada' ].head(1)
nagpada_latitude = Nagpada['Neighborhood Latitude'] 
nagpada_latitude = nagpada_latitude.iloc[0]
nagpada_longitude = Nagpada['Neighborhood Longitude'] 
nagpada_longitude = nagpada_longitude.iloc[0]
print("Nagpada latitude={} and longitude={}".format(nagpada_latitude,nagpada_longitude))


Nagpada latitude=18.9681782 and longitude=72.8286009


In [179]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Pydhoni', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Pydhoni', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Pydhoni', 'Neighborhood'] = 'Nagpada'
south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Pydhoni']
south_mumbai_venues[south_mumbai_venues.Venue == 'Kanji Manji Kothari']
#south_mumbai_venues.iloc[313]


,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
314,314,444,Nagpada,18.968178,72.828601,Kanji Manji Kothari,18.953298,72.83644,Grocery Store


In [185]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri', 'Neighborhood'] = 'Nagpada'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Dongri']
south_mumbai_venues.iloc[332:334]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
333,333,472,Nagpada,18.968178,72.828601,Delhi Zaika,18.961849,72.831166,Restaurant
334,334,473,Nagpada,18.968178,72.828601,J. J. Jalebi,18.961909,72.831670,Dessert Shop


In [189]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road']
#south_mumbai_venues.iloc[340:344]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
341,341,487,Grand Road,18.938771,72.835335,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train/Metro/Bus
342,342,507,Grand Road,18.938771,72.835335,Press Club Mumbai,18.940721,72.832469,Bar/Pub
343,343,512,Grand Road,18.938771,72.835335,Model Cafe,18.936027,72.840418,Café
344,344,513,Grand Road,18.938771,72.835335,grand hotel,18.936234,72.840724,Hotel


In [190]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road', 'Neighborhood'] = 'Nagpada'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Grand Road']
south_mumbai_venues.iloc[340:344]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
341,341,487,Nagpada,18.968178,72.828601,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train/Metro/Bus
342,342,507,Nagpada,18.968178,72.828601,Press Club Mumbai,18.940721,72.832469,Bar/Pub
343,343,512,Nagpada,18.968178,72.828601,Model Cafe,18.936027,72.840418,Café
344,344,513,Nagpada,18.968178,72.828601,grand hotel,18.936234,72.840724,Hotel


In [193]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla']
south_mumbai_venues.iloc[334:340]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
335,335,474,Byculla,18.976622,72.832794,Bhau Daji Lad Museum,18.979140,72.834449,History Museum
336,336,476,Byculla,18.976622,72.832794,Cafe Renault,18.974986,72.833758,Bar/Pub
337,337,477,Byculla,18.976622,72.832794,Gloria Restaurant,18.975485,72.833884,Asian Restaurant
338,338,479,Byculla,18.976622,72.832794,Domino's Pizza,18.978071,72.838701,Pizza Place
339,339,480,Byculla,18.976622,72.832794,Jijamata Udyan - Ranibaug,18.978606,72.834980,Zoo
340,340,481,Byculla,18.976622,72.832794,Byculla Bakery,18.977695,72.834005,Bakery


In [194]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla', 'Neighborhood'] = 'Nagpada'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Byculla']
south_mumbai_venues.iloc[334:340]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
335,335,474,Nagpada,18.968178,72.828601,Bhau Daji Lad Museum,18.979140,72.834449,History Museum
336,336,476,Nagpada,18.968178,72.828601,Cafe Renault,18.974986,72.833758,Bar/Pub
337,337,477,Nagpada,18.968178,72.828601,Gloria Restaurant,18.975485,72.833884,Asian Restaurant
338,338,479,Nagpada,18.968178,72.828601,Domino's Pizza,18.978071,72.838701,Pizza Place
339,339,480,Nagpada,18.968178,72.828601,Jijamata Udyan - Ranibaug,18.978606,72.834980,Zoo
340,340,481,Nagpada,18.968178,72.828601,Byculla Bakery,18.977695,72.834005,Bakery


### **Merge Govandi with Chembur**

In [198]:
# Get coordinates of Chembur

Chembur = south_mumbai_venues
Chembur = Chembur[Chembur.Neighborhood == 'Chembur' ].head(1)
Chembur_latitude = Chembur['Neighborhood Latitude'] 
Chembur_latitude = Chembur_latitude.iloc[0]
Chembur_longitude = Chembur['Neighborhood Longitude'] 
Chembur_longitude = Chembur_longitude.iloc[0]
print("Chembur latitude={} and longitude={}".format(Chembur_latitude,Chembur_longitude))
Chembur[Chembur.Neighborhood == 'Chembur' ].head(1)

Chembur latitude=19.0612128 and longitude=72.8975909


,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
666,666,893,Chembur,19.061213,72.897591,Le Café,19.061791,72.899479,Café


In [200]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi']
south_mumbai_venues.iloc[688:693]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
689,689,925,Govandi,19.055369,72.91507,Govandi Station,19.054849,72.915618,Platform
690,690,926,Govandi,19.055369,72.91507,Gavand Vada Pao center,19.055100,72.914816,Hot Dog Joint
691,691,927,Govandi,19.055369,72.91507,Natraj Store,19.055045,72.915612,Food & Drink Shop
692,692,928,Govandi,19.055369,72.91507,Munna Sandwich,19.055237,72.914551,Food Truck
693,693,931,Govandi,19.055369,72.91507,Deonar Park,19.051004,72.914629,Park


In [201]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi', 'Neighborhood Latitude'] = Chembur_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi', 'Neighborhood Longitude'] = Chembur_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi', 'Neighborhood'] = 'Chembur'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Govandi']
south_mumbai_venues.iloc[688:693]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
689,689,925,Chembur,19.061213,72.897591,Govandi Station,19.054849,72.915618,Platform
690,690,926,Chembur,19.061213,72.897591,Gavand Vada Pao center,19.055100,72.914816,Hot Dog Joint
691,691,927,Chembur,19.061213,72.897591,Natraj Store,19.055045,72.915612,Food & Drink Shop
692,692,928,Chembur,19.061213,72.897591,Munna Sandwich,19.055237,72.914551,Food Truck
693,693,931,Chembur,19.061213,72.897591,Deonar Park,19.051004,72.914629,Park


### **Merge Kalina with Kurla**

In [203]:
# Get coordinates of Kurla

Kurla = south_mumbai_venues
Kurla = Kurla[Kurla.Neighborhood == 'Kurla' ].head(1)
Kurla_latitude = Kurla['Neighborhood Latitude'] 
Kurla_latitude = Kurla_latitude.iloc[0]
Kurla_longitude = Kurla['Neighborhood Longitude'] 
Kurla_longitude = Kurla_longitude.iloc[0]
print("Kurla latitude={} and longitude={}".format(Kurla_latitude,Kurla_longitude))
Kurla[Kurla.Neighborhood == 'Kurla' ].head(1)

Kurla latitude=19.0652797 and longitude=72.8793805


,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
644,644,864,Kurla,19.06528,72.87938,Shri Jagrut Vinayak Temple,19.063404,72.882604,Temple


In [218]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina']
south_mumbai_venues.iloc[588:594]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
589,589,798,Kurla,19.06528,72.87938,King Chilly,19.078382,72.866350,Chinese Restaurant
590,590,799,Kurla,19.06528,72.87938,The Camp,19.077917,72.865643,Asian Restaurant
591,591,800,Kurla,19.06528,72.87938,Sadichha,19.081684,72.855762,Seafood Restaurant
592,592,801,Kurla,19.06528,72.87938,SONY CENTRE,19.077760,72.864600,Electronics Store
593,593,802,Kurla,19.06528,72.87938,king chilli kalina,19.079660,72.865732,Chinese Restaurant
594,594,803,Kurla,19.06528,72.87938,Imbiss Meating joint,19.078304,72.865934,Burger Joint


In [0]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina', 'Neighborhood Latitude'] = Kurla_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina', 'Neighborhood Longitude'] = Kurla_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina', 'Neighborhood'] = 'Kurla'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Kalina']
south_mumbai_venues.iloc[588:594]



### **Merge Malad with Andheri**

In [219]:
# Get coordinates of Andheri

Andheri = south_mumbai_venues
Andheri = Andheri[Andheri.Neighborhood == 'Andheri' ].head(1)
Andheri_latitude = Andheri['Neighborhood Latitude'] 
Andheri_latitude = Andheri_latitude.iloc[0]
Andheri_longitude = Andheri['Neighborhood Longitude'] 
Andheri_longitude = Andheri_longitude.iloc[0]
print("Chembur latitude={} and longitude={}".format(Andheri_latitude,Andheri_longitude))
Andheri[Andheri.Neighborhood == 'Andheri' ].head(1)

Chembur latitude=19.1196976 and longitude=72.8464205


,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
578,578,784,Andheri,19.119698,72.84642,Merwans Cake shop,19.1193,72.845418,Bakery


In [221]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad']
south_mumbai_venues.iloc[773:780]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
774,774,1024,Malad,19.186719,72.848588,Gupta's Bhel and Pani Puri House,19.184527,72.845946,Snack Place
775,775,1025,Malad,19.186719,72.848588,MM Mithaiwala,19.187323,72.848164,Dessert Shop
776,776,1026,Malad,19.186719,72.848588,Aditi,19.185180,72.846718,Fast Food Restaurant
777,777,1027,Malad,19.186719,72.848588,Malad Railway Station,19.187100,72.848678,Train/Metro/Bus
778,778,1028,Malad,19.186719,72.848588,platform1,19.187324,72.849162,Tourist Information Center
779,779,1029,Malad,19.186719,72.848588,aditi fast food hotel,19.185317,72.846710,Fast Food Restaurant
780,780,1031,Malad,19.186719,72.848588,Sapna Veg. Restaurant,19.192203,72.846929,Fast Food Restaurant


In [223]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad', 'Neighborhood Latitude'] = Andheri_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad', 'Neighborhood Longitude'] = Andheri_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad', 'Neighborhood'] = 'Andheri'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Malad']
south_mumbai_venues.iloc[773:780]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
774,774,1024,Andheri,19.119698,72.84642,Gupta's Bhel and Pani Puri House,19.184527,72.845946,Snack Place
775,775,1025,Andheri,19.119698,72.84642,MM Mithaiwala,19.187323,72.848164,Dessert Shop
776,776,1026,Andheri,19.119698,72.84642,Aditi,19.185180,72.846718,Fast Food Restaurant
777,777,1027,Andheri,19.119698,72.84642,Malad Railway Station,19.187100,72.848678,Train/Metro/Bus
778,778,1028,Andheri,19.119698,72.84642,platform1,19.187324,72.849162,Tourist Information Center
779,779,1029,Andheri,19.119698,72.84642,aditi fast food hotel,19.185317,72.846710,Fast Food Restaurant
780,780,1031,Andheri,19.119698,72.84642,Sapna Veg. Restaurant,19.192203,72.846929,Fast Food Restaurant


### **Merge Mahim to Bandra**

In [225]:
# Get coordinates of Bandra West

Bandra = south_mumbai_venues
Bandra = Bandra[Bandra.Neighborhood == 'Bandra West' ].head(1)
Bandra_latitude = Bandra['Neighborhood Latitude'] 
Bandra_latitude = Bandra_latitude.iloc[0]
Bandra_longitude = Bandra['Neighborhood Longitude'] 
Bandra_longitude = Bandra_longitude.iloc[0]
print("Bandra latitude={} and longitude={}".format(Bandra_latitude,Bandra_longitude))
Bandra[Bandra.Neighborhood == 'Bandra West' ].head(1)

Bandra latitude=19.0583358 and longitude=72.8302669


,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
425,425,614,Bandra West,19.058336,72.830267,Almeida Park,19.057656,72.831541,Park


In [227]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim']
south_mumbai_venues.iloc[610:618]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
611,611,822,Mahim,19.042314,72.839834,Baba Falooda,19.039995,72.841259,Dessert Shop
612,612,823,Mahim,19.042314,72.839834,Cafe Irani Chai,19.039096,72.842720,Breakfast Spot
613,613,827,Mahim,19.042314,72.839834,Domino's Pizza,19.040252,72.841394,Pizza Place
614,614,828,Mahim,19.042314,72.839834,Tazza Fast Food,19.039052,72.841866,Fast Food Restaurant
615,615,829,Mahim,19.042314,72.839834,Quarter House,19.037115,72.841462,Bar/Pub
616,616,830,Mahim,19.042314,72.839834,Xavier Institute of Engineering,19.045061,72.841716,Arcade
617,617,831,Mahim,19.042314,72.839834,Rediff.com,19.042069,72.842838,Office
618,618,832,Mahim,19.042314,72.839834,Mahim Railway Station,19.043431,72.844425,Train/Metro/Bus


In [228]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim', 'Neighborhood Latitude'] = Bandra_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim', 'Neighborhood Longitude'] = Bandra_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim', 'Neighborhood'] = 'Bandra West'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Mahim']
south_mumbai_venues.iloc[610:618]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
611,611,822,Bandra West,19.058336,72.830267,Baba Falooda,19.039995,72.841259,Dessert Shop
612,612,823,Bandra West,19.058336,72.830267,Cafe Irani Chai,19.039096,72.842720,Breakfast Spot
613,613,827,Bandra West,19.058336,72.830267,Domino's Pizza,19.040252,72.841394,Pizza Place
614,614,828,Bandra West,19.058336,72.830267,Tazza Fast Food,19.039052,72.841866,Fast Food Restaurant
615,615,829,Bandra West,19.058336,72.830267,Quarter House,19.037115,72.841462,Bar/Pub
616,616,830,Bandra West,19.058336,72.830267,Xavier Institute of Engineering,19.045061,72.841716,Arcade
617,617,831,Bandra West,19.058336,72.830267,Rediff.com,19.042069,72.842838,Office
618,618,832,Bandra West,19.058336,72.830267,Mahim Railway Station,19.043431,72.844425,Train/Metro/Bus


### **Merge Malabar Hill to Walkeshwar**

In [229]:
# Get coordinates of Bandra West

Walkeshwar = south_mumbai_venues
Walkeshwar = Walkeshwar[Walkeshwar.Neighborhood == 'Walkeshwar' ].head(1)
Walkeshwar_latitude = Walkeshwar['Neighborhood Latitude'] 
Walkeshwar_latitude = Walkeshwar_latitude.iloc[0]
Walkeshwar_longitude = Walkeshwar['Neighborhood Longitude'] 
Walkeshwar_longitude = Walkeshwar_longitude.iloc[0]
print("Walkeshwar latitude={} and longitude={}".format(Walkeshwar_latitude,Walkeshwar_longitude))
Walkeshwar[Walkeshwar.Neighborhood == 'Walkeshwar' ].head(1)

Walkeshwar latitude=18.9553434 and longitude=72.8079469


,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
247,247,317,Walkeshwar,18.955343,72.807947,Starbucks,18.956426,72.812065,Coffee Shop


In [234]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill']
south_mumbai_venues.iloc[270:280]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
270,270,345,Walkeshwar,18.955343,72.807947,HDFC Bank,18.953883,72.802378,Bank
271,271,347,Malabar Hill,18.958162,72.803366,Priyadarshini Park,18.957515,72.799614,Park
272,272,348,Malabar Hill,18.958162,72.803366,Santosh Sagar,18.961177,72.801425,Fast Food Restaurant
273,273,350,Malabar Hill,18.958162,72.803366,Country Of Origin,18.955266,72.798743,Dessert Shop
274,274,352,Malabar Hill,18.958162,72.803366,Naturals Ice Cream,18.962432,72.801471,Dessert Shop
275,275,355,Malabar Hill,18.958162,72.803366,210°Veg Gourmet Bakery,18.961828,72.802137,Bakery
276,276,357,Malabar Hill,18.958162,72.803366,Tune Up,18.954427,72.803509,Fitness/Gym/Spa
277,277,358,Malabar Hill,18.958162,72.803366,Contempory Arts,18.963036,72.803092,Arts & Crafts Store
278,278,359,Malabar Hill,18.958162,72.803366,Tart,18.963942,72.803440,Bakery
279,279,362,Kalbadevi,18.949258,72.827938,Crawford Market,18.946761,72.831225,Market


In [235]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill', 'Neighborhood Latitude'] = Walkeshwar_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill', 'Neighborhood Longitude'] = Walkeshwar_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill', 'Neighborhood'] = 'Walkeshwar'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Malabar Hill']
south_mumbai_venues.iloc[270:280]

,level_0,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
270,270,345,Walkeshwar,18.955343,72.807947,HDFC Bank,18.953883,72.802378,Bank
271,271,347,Walkeshwar,18.955343,72.807947,Priyadarshini Park,18.957515,72.799614,Park
272,272,348,Walkeshwar,18.955343,72.807947,Santosh Sagar,18.961177,72.801425,Fast Food Restaurant
273,273,350,Walkeshwar,18.955343,72.807947,Country Of Origin,18.955266,72.798743,Dessert Shop
274,274,352,Walkeshwar,18.955343,72.807947,Naturals Ice Cream,18.962432,72.801471,Dessert Shop
275,275,355,Walkeshwar,18.955343,72.807947,210°Veg Gourmet Bakery,18.961828,72.802137,Bakery
276,276,357,Walkeshwar,18.955343,72.807947,Tune Up,18.954427,72.803509,Fitness/Gym/Spa
277,277,358,Walkeshwar,18.955343,72.807947,Contempory Arts,18.963036,72.803092,Arts & Crafts Store
278,278,359,Walkeshwar,18.955343,72.807947,Tart,18.963942,72.803440,Bakery
279,279,362,Kalbadevi,18.949258,72.827938,Crawford Market,18.946761,72.831225,Market


In [0]:
test_df.to_csv("groupby_nbhood.csv")

In [0]:
south_mumbai_venues.to_csv("south_mumbai_venues_replace.csv")

## **Analyze Each Neighborhood of Mumbai.**

In [0]:
# one hot encoding
south_mumbai_onehot = pd.get_dummies(south_mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
south_mumbai_onehot['Neighborhood'] = south_mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [south_mumbai_onehot.columns[-1]] + list(south_mumbai_onehot.columns[:-1])
south_mumbai_onehot = south_mumbai_onehot[fixed_columns]

south_mumbai_onehot.shape


(851, 137)

In [0]:
#And let's examine the new dataframe size.
south_mumbai_onehot.shape  

(851, 137)

# **Group rows by neighborhood.**

### **Take mean of the frequency of occurrence of each category.**

In [0]:
south_mumbai_grouped = south_mumbai_onehot.groupby('Neighborhood').mean().reset_index()
south_mumbai_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar/Pub,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Coffee Shop,College Academic Building,College Auditorium,Comedy Club,Comfort Food Restaurant,...,Multiplex,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Platform,Plaza,Pool,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Spanish Restaurant,Sports,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tourist Information Center,Train/Metro/Bus,Vegetarian / Vegan Restaurant,Women's Store,Zoo
0,Andheri,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.100000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Apollo Bandar,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0000,0.022222,0.000000,0.000000,0.022222,0.000000,0.155556,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.088889,0.000000,0.000,0.044444,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.022222,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.022222,0.0,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bandra West,0.000000,0.000000,0.000000,0.00,0.00,0.022727,0.000000,0.0000,0.045455,0.011364,0.011364,0.068182,0.000000,0.102273,0.000000,0.000000,0.00,0.011364,0.0000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.00,0.011364,0.000000,0.102273,0.000000,0.000,0.045455,0.011364,0.000000,0.034091,0.000000,0.011364,0.011364,0.00,...,0.000000,0.00,0.000000,0.011364,0.000000,0.000,0.000000,0.0000,0.011364,0.000000,0.000000,0.034091,0.0,0.000000,0.000000,0.011364,0.011364,0.000000,0.011364,0.000000,0.000000,0.034091,0.000000,0.0,0.000000,0.011364,0.022727,0.011364,0.000000,0.011364,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,0.011364,0.000000
3,Bhuleshwar,0.000000,0.000000,0.000000,0.00,0.00,0.083333,0.000000,0.0000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.083333,0.000000,0.000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000
4,Borivali,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [0]:
## Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in south_mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = south_mumbai_grouped[south_mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Andheri----
                  venue  freq
0  Fast Food Restaurant   0.2
1            Food Court   0.1
2        Sandwich Place   0.1
3        Shop & Service   0.1
4          Burger Joint   0.1


----Apollo Bandar----
             venue  freq
0          Bar/Pub  0.16
1            Hotel  0.13
2             Café  0.09
3    Boat or Ferry  0.07
4  Fitness/Gym/Spa  0.04


----Bandra West----
                venue  freq
0                Café  0.10
1             Bar/Pub  0.10
2              Bakery  0.07
3  Chinese Restaurant  0.05
4    Asian Restaurant  0.05


----Bhuleshwar----
               venue  freq
0  Indian Sweet Shop  0.08
1             Market  0.08
2    Train/Metro/Bus  0.08
3             Arcade  0.08
4               Café  0.08


----Borivali----
                  venue  freq
0        Ice Cream Shop  0.11
1        Clothing Store  0.07
2           Snack Place  0.07
3  Fast Food Restaurant  0.07
4                  Café  0.07


----Byculla----
              venue  freq
0             

## **Let's put that into a pandas dataframe.**

## **Let's write a function to sort the venues in descending order.**

In [0]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# **Display the top 10 venues for each neighborhood.**

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = south_mumbai_grouped['Neighborhood']

for ind in np.arange(south_mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(south_mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Fast Food Restaurant,Burger Joint,Restaurant,Shop & Service,Platform,Ice Cream Shop,Bakery,Sandwich Place,Food Court,Convenience Store
1,Apollo Bandar,Bar/Pub,Hotel,Café,Boat or Ferry,Mughlai Restaurant,Chinese Restaurant,Fitness/Gym/Spa,Juice Bar,Monument / Landmark,Dessert Shop
2,Bandra West,Café,Bar/Pub,Bakery,Asian Restaurant,Chinese Restaurant,Fitness/Gym/Spa,Pizza Place,Coffee Shop,Seafood Restaurant,Snack Place
3,Bhuleshwar,Indian Sweet Shop,Café,Juice Bar,Ice Cream Shop,Hotel,Dessert Shop,Market,Snack Place,BBQ Joint,Arcade
4,Borivali,Ice Cream Shop,Fast Food Restaurant,Chinese Restaurant,Juice Bar,Sandwich Place,Snack Place,Café,Clothing Store,Department Store,Vegetarian / Vegan Restaurant


# **Cluster Neighborhood**

## **Run k-means. Adjusting number of clusters to optimum.**

In [0]:
kclusters = 10
south_mumbai_grouped_clustering = south_mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(south_mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 0, 0, 2, 2, 0, 0, 2, 0, 2], dtype=int32)

## **Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

## **Add clustering labels.**

In [0]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

## **Now merge both dataframes.**

In [0]:
south_mumbai_merged = south_mumbai_data

# merge two dataframes
south_mumbai_merged = south_mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

south_mumbai_merged.head() 


,index,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Colaba,South Mumbai,18.915091,72.825969,0,Coffee Shop,Fitness/Gym/Spa,Hotel,Chinese Restaurant,Restaurant,Diner,Bar/Pub,Furniture / Home Store,Men's Store,Café
1,1,Apollo Bandar,South Mumbai,18.918375,72.831443,0,Bar/Pub,Hotel,Café,Boat or Ferry,Mughlai Restaurant,Chinese Restaurant,Fitness/Gym/Spa,Juice Bar,Monument / Landmark,Dessert Shop
2,2,Fort,South Mumbai,18.933266,72.834515,0,Café,Chinese Restaurant,Seafood Restaurant,Coffee Shop,Dessert Shop,Bar/Pub,Plaza,Clothing Store,Lounge,Parsi Restaurant
3,3,Churchgate,South Mumbai,18.935957,72.827340,2,Sports,Fast Food Restaurant,Ice Cream Shop,Hotel,Italian Restaurant,Bar/Pub,Café,Train/Metro/Bus,Pizza Place,Coffee Shop
4,4,Nariman Point,South Mumbai,18.925951,72.823208,0,Restaurant,Theater,Hotel,Italian Restaurant,Café,Sports,Bar/Pub,Chaat Place,Coffee Shop,Mediterranean Restaurant


In [0]:
south_mumbai_merged.to_csv("south_mumbai_merged.csv")

### check **Cluster Labels** column has been added to dataframe. 

# **Visualize Clusters**

In [0]:
latitude = mumbai_latitude
longitude = mumbai_longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(south_mumbai_merged['Latitude'], south_mumbai_merged['Longitude'], south_mumbai_merged['Neighborhood'], south_mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# **This concludes the analysis of Mumbai neighborhoods . Thank You.**

In [0]:
print("asdf")

asdf
